# Udacity -- Self Driving Car Nano Degree
# Vehicle Detection

### John Mansell
---

In [6]:
%autosave 30
%matplotlib inline

Autosaving every 30 seconds


## Included Files
> [vehicleDetection.py](./vehicleDetection.py) -- The script to detect vehicles in the video  
> [WriteUP.ipynb](./WriteUp.ipynb) -- Write up for the project submission  
> [project_output.mp4](./project_output.mp4) -- Output video with the results from the project  
> [CAAR_cascade.py](./CAAR_cascade.py) -- An attempt at using HAAR cascades to detect vehicles in the video

# Vehicle Detection
> The goal of this project was to detect vehicles in a video, identify their location, and draw a bounding box around that location.

> The first step is to train a classifier to be able to identify if an image does or does not contain a car. Then, slices of an image can be fed into that classifier, and it will return (True / False) if the slice of the image contains a vehicle. If the classifier returns true, then there is a vehicle at that location and a bounding box can be drawn. If false, there is no vehicle at that location.

> This process is repeated for each frame in a video, and the final result is a new video in which all the vehicles are clearly identified.
>![](output_images/found.png)

# Gathering the Data
> I am grateful to Udacity for doing the painstaking task of providing us with over 8000 images each of cars and "not-cars".

> Because the images were provided to us already, it was fairly straightforward to read in the images from the different folders. I then created an array of 1's and 0's which corresponded to the cars and 'not-car' images. Then the data were shuffeled and separated into a training and testing set. In this case, we didn't siphon off a validation set because the validation is how well it performs on the test images and video.

# Extracting Features
> ### Feature Space
> Before we can train the classifier, its necessary to extract feature vectors for the classifier to use. Borrowing from linear algebra, a feature vector is a vector which can correlate each image to a direction and magnitude. The goal is to choose the best set of features which separate vehicle images from non-vehicle images within the vector space.
![](./output_images/feature_space.png)
> [1]

> ### HOG Features
> One of the most useful features to extract is the Histogram of Gradients (HOG) of each image. A gradient measures the direction and magnitude of changes in an image. This is useful for detecting lines in an image. Gradients are taken of different subsections of the image, and each subsection has its major lines identified. When all the lines are plotted together, their combined shape, orientation, and strength can give the outline of a car or other object. This is incredibly useful in generalizing the classifier to identify vehicles which might be in a different orientation or scale than the training data. The HOG of an image is independent of the orientation and size.
![](output_images/HOG_example2.png) 

>After testing different parameter combinations, I found 9 orientations with 8 pix_per_cell and 2 cells_per_block to be the optimum balance between acuracy and speed. I tested the different combinations by simply changing the parameters and then visually comparing the different HOG output images. Admittedly, this is not an entirely thorough assesment of the robustness of the model, but for this project it was sufficient. In an actual implimentation of a vehicle tracker, I would need to analyze the parameters, and perhaps have them dynamically set based on environment and road conditions.

> ### Spatial Binning
> Spatial binning is a way of reducing the resolution of the image. This is accomplished by grouping pixels together by their agregate color values. While higher resolution can be a good thing, reducing the resolution can have two major benefits. 
> First, it reduces the computational time necessary for each image. A 64x64 image will take less time than a 512x512 image, and most of the major features will still be present.  
> Sedond, grouping the pixels together by color can simplify the feature vector which makes it easier to classify. Spatial binning helps to smooth the difference between bright red and dark red from different parts of a car where the sun is hitting it differently.  
> Here is an example of an image of a car with the resolution reduced.
> ![](output_images/resized2.png)

> Originally I tried leaving out spatial binning and analyzing the images at full resolution. However this took too much time to analyze each image. When I implimented the spatial binning function, I used the parameters used in the lesson and saw a dramatic improvement. Visually, reducing the quality to 32x32 seemed to loose too much resolution, but it performed as well as 64x64 and much faster. So I set the spatial_size to (32x32).


> ### Color Histogram
> Another set of data that is useful is the color histogram of an image. A color histogram measures the distribution of colors in the image. One way to optimize this feature is to find a color space in which the car images are significantly different from the not-car images. In RGB color space, there's not much difference between car and not-car images, becuase there isn't a distinguishing difference between the colors that are on a car, and the colors that are on everything else. However in the YUV color space, we start to see more separation.

> Below is a plotting of the YUV color histogram of 100 images of cars and not-cars. Here you can see that especially in the Y channel of the image, cars tend to cluster together while there is a wide distribution among non-car images.
![](output_images/color_hist__cars_YUV.png)
![](output_images/color_hist__not_cars_YUV.png)

> After looking at the plots of many different color spaces, YUV seemed to show the greatest difference between car and not-car images when plotted.

# Training the Classifier
> Once the features for each image have been extracted, it's time to train the classifier. This is made incredibly simple by using the SKLearn library. The library contains functions for splitting the data into training and testing subsets, shuffling the data, initiating a LinearSVC classifier and training the classifier on the extracted data.

> The linearSVC is a support vector machine (SVM) which classifies images as car or not-car. After training on the data provided by Udacity, the linearSVC was able to produce an accuracy of 99.16%.

> Extracting the image features and training the classifier are both computationally expensive processes. Because of this, once the data were extracted and the SVC was trained, I pickled the data to use it later. This saved a lot of time since I only had to retrain the classifier if I decided to change one of the original parameters, such as color space, or number of bins in the histogram.

# Scanning for Vehicles
> After the classifier is trained, it's time to search for cars in an image. The classifier was trained on 64x64 images, so each slice was scaled to 64x64. For each of these slices I extracted the same features as were extracted in the training data. Then, the SVM predicted if the slice contained a vehicle or not. A sliding window then moved accross the image, extracting slices and feeding them to the SVM.

> To increase true detections and drown out fase positives, it was useful to use overlaping windows. The windows in my project overlaped by 50%. Overlaping by more than 50 increased the time/frame but didn't yeild better results. Overlaping by less than 50% made it more difficult to isolate false positives because true car signals aren't as strong in the heatmap.

> The different scales also effectively overlap windows, since the whole image is searched at for each scale. To determine the effective scales to use, I tried every scale from 1.1 - 2.0 incrimenting by 0.1 each time. I color coded the Hot windows by scale so that I could clearly see which scales were effective in detecting vehicles. Based on this I used as scale of 1.0, 1.2, 1.4, and 1.6. The other scales were found to be either redundent or error prone.

>![](output_images/two_windows.png)
>![](output_images/all_windows.png)

> In order to expidite this process, one technique I utilized was to extract the feature vector for the entire image, and the feed a sub-sample of that feature vector the to SVM. Effectively this is (almost) the same as extracting the feature vector for individual slices of the image but its computationally much quicker.

# Dealing with False Positives
>In each image, I scanned 1,884 windows. The SVM has an accuracy of 99.16%. So on average, I expect to have 15 false positives per image. Obviously, we don't want to identify 15 vehicles that aren't in the image. To deal with this problem, I used heatmaps. Every time there was a positive detection, I added a +1 to the pixels in that region. Since the heatmap starts at all zeros, this creates a heatmap of the most likely locations of vehicles. Since the window slices overlap eachother, this helps to make the heatmap hotter in the areas where ther is an actual vehicle. Next, I summed the heatmaps from the last 7 frames from the video. By doing this, True detections are emphasized, and the false positives are drowned out. 

>![](output_images/Combined_heatmaps2.jpg)

>Finally, I set a threshold for the heatmap, so that if the value of the pixel was less than 7 in that region, it was set to 0. Effectively, this eliminated false positives, and kept vehicles which had been detected in more than one frame.

> Another tool I used to reduce the number of false positives was to limit the search area to where the road was expected to be in the frame. This had the dual benefits of not identifying a tree as a car, and reducing the number of predictions necessary saving a lot of time.

# Results
> ### Images
![](output_images/result1.png)
![](output_images/result2.png)
![](output_images/result3.png)
![](output_images/result4.png)
![](output_images/result5.png)
![](output_images/result6.png)
Ultimately, testing on images was less robust than testing on the video, because in the video I was able to smooth together the heatmaps from multiple frames.

> ### Video
> [Project Video](project_output.mp4)



# Alternative Approaches

> ### HAAR Cascade
> I wrote a quick script to perform a HAAR cascade searching for cars in the video. The HAAR cascade was a much simpler approach, and ran pretty quickly, but the results weren't quite as good as the HOG extraction techniques used in the Udacity lesson. I'll keep this idea in mind however, since it ran more quickly and autonomous vehicles need to process frames in real time. It may be that if I can find or generate a HAAR cascade which is sufficiently effective, I could use that instead of the HOG approach to detect vehicles.

> ### MAG threshold
> In the last two udacity projects (advanced lane finding and behavioral cloning) I found that applying a magnitude of gradient threshold to the image was an effective way to get rid of a lot of the street from the image. Asphalt tends to have a lot of noise running in all directions. I tried to do this in this project as well. The idea was that if I removed the road from the image the cars would be left, and would therefore stand out against a mostly black background. However, since not all of the road was removed, the parts which were left tended to have odd unique shapes which were detected as false positives by the classifier. 

> ### Gaussian Blur
> I also tried implimenting a Gaussian blur and/or bilateral filter to the image. When combined with the spatial binning, this was too much information removed from the image and increased the rate of false positives, and decreased accuracy of detections.

# Lessons Learned

> ### Sanity Checks
> So far, one big lesson I've learned in each of the Udacity projects is to SANITY CHECK ALL THE DATA!!!!! It seems that in almost every project, I've gotten stuck somewhere because I thought the data was being processed in a particular way, but it wasn't. Because of an indentation error, I thought that all the heatmaps were being summed together, but instead only the last heatmap was being saved. This has the effect of only keeping the detections from one scale of sliding windows. Using more than one window scale effectively increased computational time and then discarded the majority of detections. The program ran without any issues, but the output wasn't what I expected. I sanity checked this by coloring the windows of detected vehicles different colors acrording to the scale used for that detection. This had the effect of showing me which scales were useful, which could be skipped, and if each scale was actually being counted toward the final detection.

>![](output_images/all_windows.png)

>### The value of global variables
> Most of the functions in the project were taken directly from the lesson. The lesson functions expect a number of vairables, some of which were set to default values. There was more than one occasion where I called a function with one parameter when training the classifier, and a different parameter when searching for cars in the image. For example, in the find_cars function I called:  

>```
ctrans_tosearch = convert_color(img, colorspace = 'HSV') ```

> When I meant to call
>```
ctrans_tosearch = convert_color(img, colorspace = colorspace)
```
> By hard coding the colorspace value, I was processing each test image in the HSV colorspace, but the calssifier had been trained on different color spaces. For the longest time I couldn't figure out why other colorspaces were performing so poorly. To correct for this, whenever possible, I called the functions with the parameters set to a global variable which I could tune at the beginning. That way they functions were called with the same parameter values when training the SVC and when searching for images.

> ### Pickle the Data
> One lesson I learned in the Behavioral Cloning project that I also implimented here was to pickle the data whenever possible. After training the classifier, I would pickle it and save it for later. If I changed any of the parameters used in training the classifier, I could re-train it and then re-pickle it. But in times when I could load a fully trained classifier, it saved a lot of computational time per iteration.

# Possible Improvements

> ### Vehicle Class
> In the walkthrough video Ryan mentioned using a Vehicle class. I made an attempt at utilizing this idea, but found it simpler to just check for vehicles directly and use a heatmap. However, I think for an actual self driving car, I would want to use a vehicle class to keep track of vehicles in view. A vehicle class would be more powerful and robust, especially for keeping track of the atributes of other vehicles, such as speed, path, etc.

> A vehicle class would also help separate overlaping detections, like when the white car goes partially behind the black car.

> ### Template Matching
> One approach that I began exploring, but ultimately didn't include was template matching. The lessons introduced the idea of template matching, which is significantly faster than searching for feature vectors, but only works if you match pretty exactly. My idea was to search for vehicles every 5 or 10 frams, but then once the vehicles have been found, only do a template matching search for the vehicles in the other 4 or 9 frames. This would save a lot of computing power, and allow for closer to real time detection. Ultimately, I'd like to combine this with the vehicle class so I could search for a vehicle I've already found, and not have to do the feature extraction search for vehicles I've already identified. Saving the computational time would be especially valuable if I also need the car to be actively scanning for traffic signs, lane lines, pedestrians, UFO's, etc.

> ### Open CV on GPU
> Much of this project utilized the openCV library. The library module can be built to utilize an NVidia GPU similar to tensorflow-GPU. Doing this would help significantly with processing time, but I wasn't able to get the build to work correctly on my machine. If I revisit this project, or impliment the strategies of this project into some later projects for the self driving car, it would be worth trouble shotting the isues to get openCV to run on the GPU.

> ### Deep Learning
> I'd like to try this project again, but using a deep learning classifier instead of a linear SVC classifier to see if it performs better in terms of accuracy and or speed. I considered using deep learning from the start, but decided to use the linear SVC for two reasons. 
>1. The lesson used a linear SVC and so implimenting the techniques from the lesson was more straightforward.   
>2. The last few projects used deep learning, and so this was a classifier type I wasn't familiar with. I was excited by the challenge of using a different type of classifier.

> ### Alternative Udacity Data
> The white car was difficult to detect when only the front half of the car is visible in the frame. I believe this is because most of the car images from the data set are of the back of the car. The alternative [car / not-car data sets](https://github.com/udacity/self-driving-car/tree/master/annotations) provided by Udacity though, include vehicles viewed from more angles and might help the classifier detect the front half of the white image.

> ### Oncoming Traffic
> The pipeline I created isn't very good at detecting oncoming traffic. This is because the oncoming traffic is small, and its velocity is pretty high within the frame. If both cars are traveling at 65 mph, oncoming traffic is coming towards the camera at an effective 130 mph. Since the heatmap threshold is designed to mute detections that aren't in the same relative position across multiple frames, oncoming traffic is filtered out. The relative position of oncoming cars within the frame changes too quickly to make it through the threshold. One way to combad this would be to combine this project with the lane-line project, and then set different threshold parameters for the right and left hand sides of the street.

# Acknowledgements

> #### Udacity
> I'd again like to thank the Udacity team for the incredible lesson quality, providing the car / not-car images, and the video walk through. Many of the functions and techniques were taken directly from the lesson or from the video walk through. It's been a great term-1. I've really enjoyed learning about self driving cars and I'm excited for term-2.

> #### SentDex
> The sentdex lesons on image processing were invaluable on learning how to process images using openCV and python. This was also where I first came accross the idea of a HAAR cascade. My attempt at a HAAR cascade was modeled after a facial recognition program he wrote in [one of his tutorials](https://youtu.be/88HdqNDQsEk). The actual "cars.xml" file was downloaded from [this](https://github.com/andrewssobral/vehicle_detection_haarcascades) GitHub repo.

> #### Image from wikipedia
>[[1] -- Feature Vector Image](https://commons.wikimedia.org/wiki/File:Kernel_yontemi_ile_veriyi_daha_fazla_dimensiyonlu_uzaya_tasima_islemi.png)

